# Data Formats

UC Davis DSI

18 July 2017

Clark Fitzgerald - @clarkfitzg


In [3]:
import pandas as pd

# Plain Text Storage

Ie. not binary

### CSV

CSV stands for "comma separated values" or "comma separated, variable
length".  This is the "lowest common denominator" since most every program
that processes tabular data can deal with CSV. I'm lumping all delimited
text files in with this.  Fixed width format is similar, since it's plain
text with one row per line.

The problem with CSV is that there are so many different conventions for 
things such as delimiters, row numbering, and escaping quotes. A robust
program needs to be capable of handling all the different options.

RFC 4180 documents a standard for CSV.

In [2]:
!cat iris.csv

"Sepal.Length","Sepal.Width","Petal.Length","Petal.Width","Species"
5.1,3.5,1.4,0.2,"setosa"
4.9,3,1.4,0.2,"setosa"
4.7,3.2,1.3,0.2,"setosa"
4.6,3.1,1.5,0.2,"setosa"
5,3.6,1.4,0.2,"setosa"
5.4,3.9,1.7,0.4,"setosa"


In [4]:
iris = pd.read_csv("iris.csv")

### JSON

Javascript Object Notation (JSON) supports two containers: key value pairs
and arrays. Like XML it's commonly used for data exchange on the internet.

Primitive objects include
- Numbers (doubles)
- Strings
- Boolean
- null

JSON translates simply and unambiguously into programming languages that
have these two containers, such as Javascript and Python, which is probably
why many prefer it over XML.

In [6]:
iris.to_json("iris.json")

In [8]:
# Using Python from the shell to pretty print the saved JSON
!python -m json.tool iris.json

{
    "Sepal.Length": {
        "0": 5.1,
        "1": 4.9,
        "2": 4.7,
        "3": 4.6,
        "4": 5.0,
        "5": 5.4
    },
    "Sepal.Width": {
        "0": 3.5,
        "1": 3.0,
        "2": 3.2,
        "3": 3.1,
        "4": 3.6,
        "5": 3.9
    },
    "Petal.Length": {
        "0": 1.4,
        "1": 1.4,
        "2": 1.3,
        "3": 1.5,
        "4": 1.4,
        "5": 1.7
    },
    "Petal.Width": {
        "0": 0.2,
        "1": 0.2,
        "2": 0.2,
        "3": 0.2,
        "4": 0.2,
        "5": 0.4
    },
    "Species": {
        "0": "setosa",
        "1": "setosa",
        "2": "setosa",
        "3": "setosa",
        "4": "setosa",
        "5": "setosa"
    }
}


In [9]:
# Make it a round trip by loading it back into Python

import json
with open("iris.json") as f:
    iris_from_json = json.load(f)
    
iris_from_json

{'Petal.Length': {'0': 1.4, '1': 1.4, '2': 1.3, '3': 1.5, '4': 1.4, '5': 1.7},
 'Petal.Width': {'0': 0.2, '1': 0.2, '2': 0.2, '3': 0.2, '4': 0.2, '5': 0.4},
 'Sepal.Length': {'0': 5.1, '1': 4.9, '2': 4.7, '3': 4.6, '4': 5.0, '5': 5.4},
 'Sepal.Width': {'0': 3.5, '1': 3.0, '2': 3.2, '3': 3.1, '4': 3.6, '5': 3.9},
 'Species': {'0': 'setosa',
  '1': 'setosa',
  '2': 'setosa',
  '3': 'setosa',
  '4': 'setosa',
  '5': 'setosa'}}

### XML

Extensible Markup Language (XML) is a flexible, general purpose
hierarchical data storage format. Use cases include exchanging data on the
internet and storing word processing documents. Schemas allow specifying
types and certain relationships among elements. Tools such as `xpath` can
efficiently query XML documents, ie. find elements of interest without
reading the entire document into memory.

In [14]:
# From Andy Hayden
# https://stackoverflow.com/a/18579083/2681019

def to_xml(df, filename=None, mode='w'):
    def row_to_xml(row):
        xml = ['<item>']
        for i, col_name in enumerate(row.index):
            xml.append('  <field name="{0}">{1}</field>'.format(col_name, row.iloc[i]))
        xml.append('</item>')
        return '\n'.join(xml)
    res = '\n'.join(df.apply(row_to_xml, axis=1))

    if filename is None:
        return res
    with open(filename, mode) as f:
        f.write(res)

pd.DataFrame.to_xml = to_xml

In [15]:
print(iris.to_xml())

<item>
  <field name="Sepal.Length">5.1</field>
  <field name="Sepal.Width">3.5</field>
  <field name="Petal.Length">1.4</field>
  <field name="Petal.Width">0.2</field>
  <field name="Species">setosa</field>
</item>
<item>
  <field name="Sepal.Length">4.9</field>
  <field name="Sepal.Width">3.0</field>
  <field name="Petal.Length">1.4</field>
  <field name="Petal.Width">0.2</field>
  <field name="Species">setosa</field>
</item>
<item>
  <field name="Sepal.Length">4.7</field>
  <field name="Sepal.Width">3.2</field>
  <field name="Petal.Length">1.3</field>
  <field name="Petal.Width">0.2</field>
  <field name="Species">setosa</field>
</item>
<item>
  <field name="Sepal.Length">4.6</field>
  <field name="Sepal.Width">3.1</field>
  <field name="Petal.Length">1.5</field>
  <field name="Petal.Width">0.2</field>
  <field name="Species">setosa</field>
</item>
<item>
  <field name="Sepal.Length">5.0</field>
  <field name="Sepal.Width">3.6</field>
  <field name="Petal.Length">1.4</field>
  <fiel

# Binary Storage

### HDF5

Hierarchical Data Format (HDF5) was originally designed to store and
process large amounts of scientific data. It has a rich data model and
includes a mature software implementation written in C. It stores data in a
binary form and scales up to HPC use cases.

netCDF uses HDF5.

In [30]:
iris.to_hdf("iris.hdf", key = "iris")

In [31]:
!h5dump iris.hdf

HDF5 "iris.hdf" {
GROUP "/" {
   ATTRIBUTE "CLASS" {
      DATATYPE  H5T_STRING {
         STRSIZE 5;
         STRPAD H5T_STR_NULLTERM;
         CSET H5T_CSET_UTF8;
         CTYPE H5T_C_S1;
      }
      DATASPACE  SCALAR
      DATA {
      (0): "GROUP"
      }
   }
   ATTRIBUTE "PYTABLES_FORMAT_VERSION" {
      DATATYPE  H5T_STRING {
         STRSIZE 3;
         STRPAD H5T_STR_NULLTERM;
         CSET H5T_CSET_UTF8;
         CTYPE H5T_C_S1;
      }
      DATASPACE  SCALAR
      DATA {
      (0): "2.1"
      }
   }
   ATTRIBUTE "TITLE" {
      DATATYPE  H5T_STRING {
         STRSIZE 1;
         STRPAD H5T_STR_NULLTERM;
         CSET H5T_CSET_UTF8;
         CTYPE H5T_C_S1;
      }
      DATASPACE  NULL
      DATA {
      }
   }
   ATTRIBUTE "VERSION" {
      DATATYPE  H5T_STRING {
         STRSIZE 3;
         STRPAD H5T_STR_NULLTERM;
         CSET H5T_CSET_UTF8;
         CTYPE H5T_C_S1;
      }
      DATASPACE  SCALAR
      DATA {
      (0): "1.0"
      }
   }
   GROUP "iris" {
      ATT